In [1]:
# Summary: the code is FEniCS-based implementation of Stokes and contiuum equations 
#    with mechanical anisotropy described in Maulhaus et al. (2002). 
# Part of the code is adopted from the FEniCS Stokes-iterative demo, whose link is provided below:
#    https://fenicsproject.org/olddocs/dolfin/1.3.0/python/demo/documented/stokes-iterative/python/documentation.html

# Now version 1.1.9. 
# Created on 08/02/2021. Last modfied on 06/29/2021. 
# Author: Dunyu Liu (dliu@ig.utexas.edu) 

In [2]:
from dolfin import *
import numpy as np
import ufl as ufl
import sys
import math
import matplotlib.pyplot as plt
from lib import *
from prepare_case import *

%matplotlib inline
#sys.setrecursionlimit(1500)

# Test for PETSc or Epetra
if not has_linear_algebra_backend("PETSc") and not has_linear_algebra_backend("Epetra"):
    info("DOLFIN has not been configured with Trilinos or PETSc. Exiting.")
    exit()

if not has_krylov_solver_preconditioner("amg"):
    info("Sorry, this code is only available when DOLFIN is compiled with AMG "
	 "preconditioner, Hypre or ML.");
    exit()

In [3]:
case = 30 
# case 30 for 3D box with a horziontal anisotropic layer.
# case 31 for 3D box with vertical SAF fault zone parallel to boundaries.
# case 32 for 
date_stamp = '20220629' # Date stamp is used in the output folder name.

In [4]:
def stokes(delta, eta_strong, eta_weak, msh_path = None, msh_name = None, n = 10):
    
    # Use function prepare_case to generate key model specific data.
    mesh, boundaries, mf, bcs, f, bound_name_list, dim, W = prepare_case(case, msh_path, msh_name, n)
    #------- Case specific modifications end here --------
    
    anis_domain = bound_name_list['anis_domain']
    iso_domain = bound_name_list['iso_domain']
    
    ds = Measure("ds")(domain=mesh, subdomain_data = boundaries)
    
    if eta_strong>eta_weak:
        hete = 1
        iso = False
    elif eta_strong == eta_weak: 
        hete = 0
        iso = True

    # Define eta and eta_s.
    # eta is the strong viscosity in the anisotropic cases or the viscosity of isotropic case.
    # eta_s is the weak viscosity in the anisotropic cases.
    if iso == True:
        eta = K(mf, eta_weak, eta_strong, anis_domain, iso_domain, degree=0)
    else:
        eta = eta_strong
    eta_s = K(mf, eta_weak, eta_strong, anis_domain, iso_domain, degree=0) 
    
    # Define functions -----------------------------------
    def epsilon(v):
        return sym(nabla_grad(v))
    # Define stress
    def sigma(v,p,eta):
        return 2*eta*sym(nabla_grad(v))-(-p)*Identity(dim)
    
    # https://www.continuummechanics.org/principalstressesandstrains.html
    # calc_princ is derived from the above link.
    def calc_princ(sigma):
        thetap = atan(2*sigma[0,1]/(sigma[0,0] - sigma[1,1]))
        thetap = thetap/math.pi*180/2

        sigma1 = (sigma[0,0] + sigma[1,1])/2 + sqrt(pow((sigma[0,0]/2 - sigma[1,1]/2),2) + sigma[0,1]*sigma[0,1])
        sigma3 = (sigma[0,0] + sigma[1,1])/2 - sqrt(pow((sigma[0,0]/2 - sigma[1,1]/2),2) + sigma[0,1]*sigma[0,1])

        return thetap, sigma1, sigma3
    
    # Define anisotropic viscosity constitutive relation
    # Based on Maulhaus et al. (2002). 
    def sigma_anisotropic(v, eta, eta_s, dim):
        if dim == 2:
            n = np.zeros((1,dim))
            n[0][0] = norm1
            n[0][1] = norm3       
        elif dim == 3:
            n = np.zeros((1,dim))
            n[0][0] = norm1
            n[0][1] = norm2
            n[0][2] = norm3
        C_lambda = np.zeros((dim, dim, dim, dim))

        componentList = []
        for i in range(dim):
            componentList += [[],]
            for j in range(dim):
                componentList[i] += [[],]
                for k in range(dim):
                    componentList[i][j] += [[],]
                    for l in range(dim):
                        componentList[i][j][k] += [[],]    

        for i in range(dim):
            for j in range(dim):
                for k in range(dim):
                    for l in range(dim):
                        a = 0
                        b = 0
                        c = 0
                        d = 0
                        e = n[0][i]*n[0][j]*n[0][k]*n[0][l]
                        if l == j :
                            a = n[0][i]*n[0][k]
                        if i == l :
                            b = n[0][j]*n[0][k]
                        if k == j :
                            c = n[0][i]*n[0][l]
                        if i == k :
                            d = n[0][j]*n[0][l]
                        componentList[i][j][k][l] = 2*(eta_s - eta) * ((a + b + c + d)/2 - 2*e)
                        #C_lambda[i][j][k][l] = (a + b + c + d)/2 - 2*e

        C = as_tensor(componentList)
        vtmp = as_tensor(epsilon(v))
        i, j, k, l = ufl.indices(4)
        C1 = ufl.as_tensor(C[i,j,k,l]*vtmp[k,l],(i,j))
        C2 = 2*eta*sym(nabla_grad(v))
        C3 = C1 + C2
        return C3

    # Define variational problem
    (u, p) = TrialFunctions(W)
    (v, q) = TestFunctions(W)
    
    theta = 90 + delta
    norm1 = cos(theta/180*math.pi)
    norm2 = sin(theta/180*math.pi)
    norm3 = 0    

    print('Normal vectors for anisotrpy are ', norm1, norm2, norm3, 'in this scenario ...')
    
    if iso:
        theta2 = -10000
        nametag = '_isotropic' # Appended in the result files.
        a = inner(grad(u), eta*grad(v))*dx + div(v)*p*dx + q*div(u)*dx
        L = inner(f, v)*dx
        # b is used to build the preconditioner matrix
        b = inner(grad(u), eta*grad(v))*dx + p*q*dx
    else:
        nametag = '_anisotropic'
        a1 = inner(sigma_anisotropic(u,eta,eta_s,dim), grad(v))*dx
        a2 = div(v)*p*dx + q*div(u)*dx
        a = a1 + a2
        L = inner(f, v)*dx
        # b is used to build the preconditioner matrix
        b = inner(grad(u), 2*eta*grad(v))*dx + p*q*dx    

    # Assemble the system
    A, bb = assemble_system(a, L, bcs)
    # Assemble the preconditioner system
    P, btmp = assemble_system(b, L, bcs)
    print('Assembling the system and preconditioner ...')

    # Create Krylov solver and AMG preconditioner
    solver = KrylovSolver("minres", "amg") # best.
    #solver = KrylovSolver("gmres", "amg") # second best.
    #solver = KrylovSolver("tfqmr", "amg") # least

    solver.parameters["relative_tolerance"] = 1.0e-9
    solver.parameters["absolute_tolerance"] = 1.0e-15
    solver.parameters["divergence_limit"] = 1.0e4
    solver.parameters["maximum_iterations"] = 7000
    solver.parameters["error_on_nonconvergence"] = True
    solver.parameters["nonzero_initial_guess"] = False
    solver.parameters["report"] = True
    solver.parameters["monitor_convergence"] = True
    # Associate operator (A) and preconditioner matrix (P)
    solver.set_operators(A, P)

    # Solve
    U = Function(W)
    
    print('Solving the system, be patient...')

    solver.solve(U.vector(), bb)
    
    # Get sub-functions
    u, p = U.split()

    # Calculate stress and strain-rate tensors
    Vsig = TensorFunctionSpace(mesh, "DG", degree=0) # Set DG0 for stress & strain rate
    sig = Function(Vsig, name="Stress")
    strain_rate = Function(Vsig, name="Strain")

    # Project stresses to Vsig space.
    if iso==True:
        sig.assign(project(sigma(u,p,eta), Vsig))
    else: 
        sig.assign(project(sigma_anisotropic(u,eta,eta_s,dim)-(-p)*Identity(dim), Vsig))

    strain_rate.assign(project(epsilon(u), Vsig))

    print('Writing out the results ...')
    # Save solution in xdmf format
    ufile_pvd = XDMFFile("../res/case"+ str(case) + "/" + str(date_stamp) + "/velocity_theta"+str(delta) + "_hetero_"+ str(hete) + ".xdmf")
    ufile_pvd.write(u)
    pfile_pvd = XDMFFile("../res/case"+ str(case) + "/" + str(date_stamp) + "/pressure_theta"+str(delta) + "_hetero_"+ str(hete) + ".xdmf")
    pfile_pvd.write(p)
    pfile_pvd = XDMFFile("../res/case"+ str(case) + "/" + str(date_stamp) + "/stress_theta"+str(delta) + "_hetero_"+ str(hete) + ".xdmf")
    pfile_pvd.write(sig)
    pfile_pvd = XDMFFile("../res/case" + str(case) + "/" + str(date_stamp) + "/strain_rate_theta"+str(delta) + "_hetero_"+ str(hete) + ".xdmf")
    pfile_pvd.write(strain_rate)
    
    print('Finished ...')

In [ ]:
# theta is the angle of the layer normal to the horizontal axis in the square model.
# theta should be >90 and <180.
# delta = theta - 90.

if case == 30: 
    n = 10
    eta_strong, eta_weak = 1, 0.1
    msh_path = None
    msh_name = None
    for delta in np.arange(0,92.5,2.5):
        stokes(delta, eta_strong, eta_weak, msh_path, msh_name, n)
        
if case == 31:
    n = 10
    eta_strong, eta_weak = 1, 0.1
    msh_path = None
    msh_name = None
    for delta in np.arange(0,92.5,12.5):
        stokes(delta, eta_strong, eta_weak, msh_path, msh_name, n)
        
if case == 32:
    msh_path = "../msh/Cascadia_schist_mesh_case32/"
    msh_name = "cascadia_thrust"
    delta = 30
    eta_strong, eta_weak = 1, 0.1
    stokes(delta, eta_strong, eta_weak, msh_path, msh_name, n=10)

Simulating Case 30: 3D box model with a horizontal anisotropic layer.
Defining boundary conditions ...
Normal vectors for anisotrpy are  6.123233995736766e-17 1.0 0 in this scenario ...
Calling FFC just-in-time (JIT) compiler, this may take some time.
Assembling the system and preconditioner ...
Solving the system, be patient...
Solving linear system of size 26300 x 26300 (PETSc Krylov solver).
  0 KSP preconditioned resid norm 3.012297065952e+01 true resid norm 1.288488347725e+02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP preconditioned resid norm 1.136870983946e+00 true resid norm 7.356974418022e-01 ||r(i)||/||b|| 5.709771788789e-03
  2 KSP preconditioned resid norm 4.527827120663e-02 true resid norm 2.496525594774e-02 ||r(i)||/||b|| 1.937561638941e-04
  3 KSP preconditioned resid norm 9.076585091292e-03 true resid norm 4.398106164780e-03 ||r(i)||/||b|| 3.413384507936e-05
  4 KSP preconditioned resid norm 8.514016822678e-03 true resid norm 3.270078321578e-03 ||r(i)||/||b|| 2.537918427

Writing out the results ...
Finished ...
Simulating Case 30: 3D box model with a horizontal anisotropic layer.
Defining boundary conditions ...
Normal vectors for anisotrpy are  -0.04361938736533589 0.9990482215818578 0 in this scenario ...
Calling FFC just-in-time (JIT) compiler, this may take some time.
Assembling the system and preconditioner ...
Solving the system, be patient...
Solving linear system of size 26300 x 26300 (PETSc Krylov solver).
  0 KSP preconditioned resid norm 3.012297065952e+01 true resid norm 1.288488347725e+02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP preconditioned resid norm 1.136914115527e+00 true resid norm 7.356963517580e-01 ||r(i)||/||b|| 5.709763328920e-03
  2 KSP preconditioned resid norm 4.575742401100e-02 true resid norm 2.515641460043e-02 ||r(i)||/||b|| 1.952397524188e-04
  3 KSP preconditioned resid norm 9.798464039219e-03 true resid norm 4.580002530961e-03 ||r(i)||/||b|| 3.554554869703e-05
  4 KSP preconditioned resid norm 9.186676084614e-03 true r

Writing out the results ...
Finished ...
Simulating Case 30: 3D box model with a horizontal anisotropic layer.
Defining boundary conditions ...
Normal vectors for anisotrpy are  -0.08715574274765824 0.9961946980917455 0 in this scenario ...
Calling FFC just-in-time (JIT) compiler, this may take some time.
Assembling the system and preconditioner ...
Solving the system, be patient...
Solving linear system of size 26300 x 26300 (PETSc Krylov solver).
  0 KSP preconditioned resid norm 3.012297065952e+01 true resid norm 1.288488347725e+02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP preconditioned resid norm 1.137043292271e+00 true resid norm 7.357023328026e-01 ||r(i)||/||b|| 5.709809748003e-03
  2 KSP preconditioned resid norm 4.716125523838e-02 true resid norm 2.570864645768e-02 ||r(i)||/||b|| 1.995256418351e-04
  3 KSP preconditioned resid norm 1.167056776442e-02 true resid norm 5.123949932596e-03 ||r(i)||/||b|| 3.976714218366e-05
  4 KSP preconditioned resid norm 1.080214379489e-02 true r

Writing out the results ...
Finished ...
Simulating Case 30: 3D box model with a horizontal anisotropic layer.
Defining boundary conditions ...
Normal vectors for anisotrpy are  -0.13052619222005138 0.9914448613738105 0 in this scenario ...
Calling FFC just-in-time (JIT) compiler, this may take some time.
Assembling the system and preconditioner ...
Solving the system, be patient...
Solving linear system of size 26300 x 26300 (PETSc Krylov solver).
  0 KSP preconditioned resid norm 3.012297065952e+01 true resid norm 1.288488347725e+02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP preconditioned resid norm 1.137257507953e+00 true resid norm 7.357154098091e-01 ||r(i)||/||b|| 5.709911239076e-03
  2 KSP preconditioned resid norm 4.940069852896e-02 true resid norm 2.659654488540e-02 ||r(i)||/||b|| 2.064166504289e-04
  3 KSP preconditioned resid norm 1.416811105356e-02 true resid norm 5.908896006819e-03 ||r(i)||/||b|| 4.585913421145e-05
  4 KSP preconditioned resid norm 1.280691988171e-02 true r

Writing out the results ...
Finished ...
Simulating Case 30: 3D box model with a horizontal anisotropic layer.
Defining boundary conditions ...
Normal vectors for anisotrpy are  -0.1736481776669303 0.984807753012208 0 in this scenario ...
Calling FFC just-in-time (JIT) compiler, this may take some time.
Assembling the system and preconditioner ...
Solving the system, be patient...
Solving linear system of size 26300 x 26300 (PETSc Krylov solver).
  0 KSP preconditioned resid norm 3.012297065952e+01 true resid norm 1.288488347725e+02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP preconditioned resid norm 1.137555093769e+00 true resid norm 7.357355285437e-01 ||r(i)||/||b|| 5.710067381230e-03
  2 KSP preconditioned resid norm 5.235249412281e-02 true resid norm 2.778310734959e-02 ||r(i)||/||b|| 2.156256003296e-04
  3 KSP preconditioned resid norm 1.692021309607e-02 true resid norm 6.827060462102e-03 ||r(i)||/||b|| 5.298503843017e-05
  4 KSP preconditioned resid norm 1.481825630431e-02 true res

...

Writing out the results ...
Finished ...
Simulating Case 30: 3D box model with a horizontal anisotropic layer.
Defining boundary conditions ...
Normal vectors for anisotrpy are  -0.5735764363510458 0.819152044288992 0 in this scenario ...
Calling FFC just-in-time (JIT) compiler, this may take some time.
Assembling the system and preconditioner ...
Solving the system, be patient...
Solving linear system of size 26300 x 26300 (PETSc Krylov solver).
  0 KSP preconditioned resid norm 3.012297065952e+01 true resid norm 1.288488347725e+02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP preconditioned resid norm 1.144318594473e+00 true resid norm 7.362577850294e-01 ||r(i)||/||b|| 5.714120630812e-03
  2 KSP preconditioned resid norm 9.773415690118e-02 true resid norm 4.758342284498e-02 ||r(i)||/||b|| 3.692964932823e-04
  3 KSP preconditioned resid norm 3.881785391071e-02 true resid norm 1.610740791583e-02 ||r(i)||/||b|| 1.250101170435e-04
  4 KSP preconditioned resid norm 2.461386531402e-02 true res

Writing out the results ...
Finished ...
Simulating Case 30: 3D box model with a horizontal anisotropic layer.
Defining boundary conditions ...
Normal vectors for anisotrpy are  -0.6087614290087207 0.7933533402912352 0 in this scenario ...
Calling FFC just-in-time (JIT) compiler, this may take some time.
Assembling the system and preconditioner ...
Solving the system, be patient...
Solving linear system of size 26300 x 26300 (PETSc Krylov solver).
  0 KSP preconditioned resid norm 3.012297065952e+01 true resid norm 1.288488347725e+02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP preconditioned resid norm 1.145257544106e+00 true resid norm 7.363309516223e-01 ||r(i)||/||b|| 5.714688479120e-03
  2 KSP preconditioned resid norm 1.024401833858e-01 true resid norm 4.974631028296e-02 ||r(i)||/||b|| 3.860827330787e-04
  3 KSP preconditioned resid norm 4.018710490186e-02 true resid norm 1.687952078884e-02 ||r(i)||/||b|| 1.310025101791e-04
  4 KSP preconditioned resid norm 2.500860360321e-02 true re

Writing out the results ...
Finished ...
Simulating Case 30: 3D box model with a horizontal anisotropic layer.
Defining boundary conditions ...
Normal vectors for anisotrpy are  -0.6427876096865394 0.766044443118978 0 in this scenario ...
Calling FFC just-in-time (JIT) compiler, this may take some time.
Assembling the system and preconditioner ...
Solving the system, be patient...
Solving linear system of size 26300 x 26300 (PETSc Krylov solver).
  0 KSP preconditioned resid norm 3.012297065952e+01 true resid norm 1.288488347725e+02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP preconditioned resid norm 1.146218067390e+00 true resid norm 7.364053298505e-01 ||r(i)||/||b|| 5.715265730970e-03
  2 KSP preconditioned resid norm 1.070350408000e-01 true resid norm 5.187292834718e-02 ||r(i)||/||b|| 4.025874850850e-04
  3 KSP preconditioned resid norm 4.143807744251e-02 true resid norm 1.762213964711e-02 ||r(i)||/||b|| 1.367659993063e-04
  4 KSP preconditioned resid norm 2.536646571519e-02 true res

Writing out the results ...
Finished ...
Simulating Case 30: 3D box model with a horizontal anisotropic layer.
Defining boundary conditions ...
Normal vectors for anisotrpy are  -0.6755902076156602 0.7372773368101241 0 in this scenario ...
Calling FFC just-in-time (JIT) compiler, this may take some time.
Assembling the system and preconditioner ...
Solving the system, be patient...
Solving linear system of size 26300 x 26300 (PETSc Krylov solver).
  0 KSP preconditioned resid norm 3.012297065952e+01 true resid norm 1.288488347725e+02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP preconditioned resid norm 1.147192817194e+00 true resid norm 7.364802112033e-01 ||r(i)||/||b|| 5.715846887586e-03
  2 KSP preconditioned resid norm 1.114989278595e-01 true resid norm 5.395285875776e-02 ||r(i)||/||b|| 4.187298926920e-04
  3 KSP preconditioned resid norm 4.258092291378e-02 true resid norm 1.833747525148e-02 ||r(i)||/||b|| 1.423177422121e-04
  4 KSP preconditioned resid norm 2.569356758292e-02 true re

Writing out the results ...
Finished ...
Simulating Case 30: 3D box model with a horizontal anisotropic layer.
Defining boundary conditions ...
Normal vectors for anisotrpy are  -0.7071067811865475 0.7071067811865476 0 in this scenario ...
Calling FFC just-in-time (JIT) compiler, this may take some time.
Assembling the system and preconditioner ...
Solving the system, be patient...
Solving linear system of size 26300 x 26300 (PETSc Krylov solver).
  0 KSP preconditioned resid norm 3.012297065952e+01 true resid norm 1.288488347725e+02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP preconditioned resid norm 1.148174353351e+00 true resid norm 7.365548960752e-01 ||r(i)||/||b|| 5.716426519307e-03
  2 KSP preconditioned resid norm 1.158145058783e-01 true resid norm 5.597695962232e-02 ||r(i)||/||b|| 4.344390053753e-04
  3 KSP preconditioned resid norm 4.362463062358e-02 true resid norm 1.902691953706e-02 ||r(i)||/||b|| 1.476685417502e-04
  4 KSP preconditioned resid norm 2.599389731409e-02 true re